In [1]:
'''
Precomputation of the B-smooth primes table.
'''
B = 2^32
pt = prime_range(B+1)

In [28]:
'''
Cell containing function definitions.
'''
import multiprocessing
from sage.all import factor


verbose = 0  # Change to 1 for more informations to be printed
def factorize(n, queue):
    '''
    Function used to run factorization and put results into queue.
    
    Parameters
    ----------
    n : int
        Integer to be factorized.
    queue : multiprocessing.queues.Queue
        Multiprocesing queue object.
    '''
    try:
        result = factor(n)
        queue.put(result)
    except Exception as e:
        queue.put(f"Error: {e}")

def safe_factor(n, timeout=5):
    '''
    Function used to run factorization with time limit.
    
    Parameters
    ----------
    n : int
        Integer to be factorized.
    timeout : int
        Time constraint for factorization step. Default: 5
    
    Returns
    -------
    list or str
        List of tuples composed of factor and exponent or error
        message e.g. when n is 0 or factorization reached time limit.
    '''
    queue = multiprocessing.Queue()
    process = multiprocessing.Process(target=factorize, args=(n, queue))
    
    process.start()
    process.join(timeout)

    if process.is_alive():
        process.terminate()
        process.join()
        return f"Timeout for {n}"
    
    return queue.get()

def TD(x, pt):
    '''
    Trial division.
    
    This function is used to reduce number of candidates reaching
    costful full factorization step by omitting these having small
    divisors congruent to 3 mod 4 with odd exponent.
    
    Parameters
    ----------
    x : int
        Integer to be check through trial division.
    pt : list
        List of primes used in trial division. 
    
    Returns
    -------
    bool
        False if candidate must be discarded, True otherwise.
    int
        Value remaining after trial division.
    list
        List of tuples composed of factor and exponent analogously
        to return of factor() function.
    '''
    r = x
    t = []
    for p in pt:
        i = 0
        while r % p == 0:
            r //= p
            i += 1
        if i > 0:
            t.append((p,i))
        if p%4 == 3 and i%2 != 0:
            if verbose >= 1:
                print(f'has divisior p equiv 3 mod 4 with odd exponent (!)')
            return False, 1, t
        if r == 1:
            return True, 1, t
    return True, r, t

def OddCyclicSumOfSquares(n, factexpl):
    '''
    Decomposing integer into sum of two squares.
    
    Parameters
    ----------
    n : int
        Integer to be decomposed.
    factexpl : list
        List of prime factors of square-free part of n. 
    
    Returns
    -------
    bool
        False if factexpl contains prime congruent to 3 mod 4,
        True otherwise.
    int
        First component of the sum of squares.
    int
        Second component of the sum of squares.
    '''
    fac = factexpl
    if not set([f % 4 for f in fac]) == set([1]):
        return False, 0, 0
    else:
        Z = ZZ[I]
        prod = 1
        for f in fac:
            p = f
            F = GF(p)
            z = F.random_element()
            while not z^((p-1)//2) == F(-1):
                z = F.random_element()
            z = z^((p-1)//4)
            z = int(z)
            prod *= gcd(Z(p), Z(z+I))
        u = [int(e) for e in prod]
        if u[0]%2==1: 
            u1 = u[0]
            u2 = u[1]//2 
        else:
            u1 = u[1]
            u2 = u[0]//2
        return True, abs(u1), abs(u2)

def prod(tab):
    '''
    Product of elements of the sequence.
    
    Parameters
    ----------
    tab : list or tuple or set
        Sequence of elements to be multiplied.
    
    Returns
    -------
    int
        Product of elements or 1 if seqence was empty.
    
    It is worth noting that product of empty sequence should 
    be 0 or cause raising of an exception. However in this case
    it is used only to compute square root of squared part of c
    as a cofactor w s.t. 2^a-3^b=c=w^2(u^2+(2v)^2)=(uw)^2+(2vw)^2.
    '''
    res = 1
    for t in tab:
        res *= t
    return res

In [30]:
'''
Main computation cell. It is assumed that in the working directory
contains directory named results as the results will be stored
there. It provides that main directory is not messed with a lot
of txt files.
'''
import time


a = 382 
b = 241 #Change to desired value
while True:
    ea = a
    eb = b
    nf = True
    t_start = time.time()
    while nf:
        c = 2^ea-3^eb
        if c < 0:
            ea += 1
            continue
        if verbose >= 1:
            print(f'(b,a)=({b},{ea}): ', end='')
        d, r, t = TD(c, pt)
        if not d:
            ea += 1
            continue
        if r%4 == 3:
            if verbose >= 1:
                print('reminder equiv 3 mod 4 (!)')
            ea += 1
            continue
        if verbose >= 1:
            print(f'may be correct (?)')
        f = safe_factor(r, timeout=1500) if r != 1 else [(1,1)]
        if verbose >= 1:
            print(f'\tsafe_factor result: {f if not f[0] == "T" else "Timeout (!)"}')
        if f[0] == "T":
            ea += 1
            continue
        else:
            if True in [fac[0]%4==3 and fac[1]%2==1 for fac in f]:
                if verbose >= 1:
                    print(f'has reminder divisor equiv 3 mod 4 with odd exponent (!)')
                ea += 1
                continue
            else:
                nf = False
                print(f'Found fitting candidate a = {ea} for b = {eb} in {(time.time()-t_start):.2f}s')
                factors = [_[0] for _ in t] + [_[0] for _ in f]
                exponents = [_[1] for _ in t] + [_[1] for _ in f]
                with open(f"results/pair_{eb}_{ea}.txt", "w") as file:
                    file.write(f"{str(eb)}\n{str(ea)}\n{[str(fac) for fac in factors]}\n{[str(exp) for exp in exponents]}\n")
    b += 2
    while 3^b > 2^a:
        a += 1

Found fitting candidate a = 385 for b = 241 in 1364.21s
Found fitting candidate a = 390 for b = 243 in 1401.64s
Found fitting candidate a = 404 for b = 245 in 3058.21s
Found fitting candidate a = 397 for b = 247 in 1105.12s
Found fitting candidate a = 403 for b = 249 in 914.71s
Found fitting candidate a = 404 for b = 251 in 1634.88s
Found fitting candidate a = 401 for b = 253 in 68.03s
Found fitting candidate a = 407 for b = 255 in 1111.23s
Found fitting candidate a = 408 for b = 257 in 68.33s
Found fitting candidate a = 421 for b = 259 in 1803.82s
Found fitting candidate a = 414 for b = 261 in 125.15s
Found fitting candidate a = 429 for b = 263 in 213.41s
Found fitting candidate a = 449 for b = 265 in 7573.32s
Found fitting candidate a = 430 for b = 267 in 70.49s
Found fitting candidate a = 434 for b = 269 in 1635.30s
Found fitting candidate a = 449 for b = 271 in 831.91s
Found fitting candidate a = 437 for b = 273 in 1705.43s
Found fitting candidate a = 437 for b = 275 in 1012.83s
Fo

KeyboardInterrupt: 

/tmp/ipykernel_187/1492855451.py:21: RuntimeWarning: cypari2 leaked 1194808 bytes on the PARI stack
  result = factor(n)
Process Process-236:
Traceback (most recent call last):
  File "/home/kozga/sage/sage-9.5/local/var/lib/sage/venv-python3.9.9/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/kozga/sage/sage-9.5/local/var/lib/sage/venv-python3.9.9/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_187/1492855451.py", line 21, in factorize
    result = factor(n)
  File "/home/kozga/sage/sage-9.5/local/var/lib/sage/venv-python3.9.9/lib/python3.9/site-packages/sage/arith/misc.py", line 2608, in factor
    return m(proof=proof, algorithm=algorithm, int_=int_,
  File "sage/rings/integer.pyx", line 3921, in sage.rings.integer.Integer.factor (build/cythonized/sage/rings/integer.c:25397)
    F = factor_using_pari(n, int_=int_, debug_level=verbose, proof=proof)
  File "

In [32]:
'''
This cell gathers results from the forementioned directory 
and presents them in human-readable form.
'''
import glob
ba_tab = []
facs = []
for b in range(241, 370, 2):
    for fn in glob.glob(f"results/pair_{b}_*.txt"):
        with open(fn) as f:
            _ = [ __[:-1] for __ in f]
            _factors = eval(_[2])
            _exponents = eval(_[3])
            print(f'pair ({_[0]}, {_[1]}): 2^{_[1]}-3^{_[0]} = {" * ".join(_factors) if set(_exponents) == {"1"}  else " * ".join([f"{_factors[i]}^{_exponents[i]}" for i in range(len(_factors))])}')
            ba_tab.append((int(_[0]), int(_[1])))
            facs.append((_factors, _exponents))
print('Completed')

pair (241, 385): 2^385-3^241 = 18461 * 27896761 * 23791170096424437412603475793049 * 5640992388863695030771167784482066743193271233755823270401329620148314001
pair (243, 390): 2^390-3^243 = 1021 * 91957 * 9774433701012345408441378880093 * 2652861338171734376902641393014975376620250666568319226482337785823478211640457
pair (245, 404): 2^404-3^245 = 13 * 3178093334046781829682375427108685941717693846816610278426136921359647694889662738701141105579464909063569202466686838721
pair (247, 397): 2^397-3^247 = 5 * 9413 * 6708145622426447170775610321532386312201405168147661050608836222099302760384862439037164551525844313120744099082429
pair (249, 403): 2^403-3^249 = 5^2 * 601^1 * 6673^1 * 205406571996101239716287143242874369103799572108833433431247904846684025162352721679645958472818765811365239925789^1
pair (251, 404): 2^404-3^251 = 13 * 3134149754233857506748108247533412112782423954324625476383032210762900247474426124969629428426421740772706510632951362713
pair (253, 401): 2^401-3^253 = 53 * 

In [ ]:
'''
This cell gathers result from the forementioned directory and
computes uv-representation based on factorization of c. Results
are printed and stored in main directory in a python file.
'''
file = open("uvExpanded.py", "w")
file.write('uvtable = [\n')
for j in range(len(ba_tab)):
    r = 2^ba_tab[j][1]-3^ba_tab[j][0]
    print(f'2^{ba_tab[j][1]}-3^{ba_tab[j][0]} = ', end='')
    f = []
    f_even_exponents = []
    for i in range(len(facs[j][0])):
        if int(facs[j][0][i]) == 1:
            continue
        if int(facs[j][1][i]) == 1:
            f.append(int(facs[j][0][i]))
        elif int(facs[j][1][i]) % 2 == 0:
            f_even_exponents.append((int(facs[j][0][i]), int(facs[j][1][i])))
        else:
            f.append(int(facs[j][0][i]))
            f_even_exponents.append((int(facs[j][0][i]), int(facs[j][1][i])-1))
    _, u, v = OddCyclicSumOfSquares(n, f)
    cofactor = prod([el[0]^(el[1]//2) for el in f_even_exponents])
    u *= cofactor
    v *= cofactor
    print(f"{u}^2 + (2*{v})^2")
    assert r == u^2+(2*v)^2
    if j + 1 != len(ba_tab):
        file.write(f"[{ba_tab[j][0]}, {ba_tab[j][1]}, {u}, {v}],\n")
    else:
        file.write(f"[{ba_tab[j][0]}, {ba_tab[j][1]}, {u}, {v}]]")
file.close()